Set up all the modules we need

In [ ]:
import RPi.GPIO as GPIO
import time
import pigpio
import numpy as np

Making the numpy file for the shot

In [ ]:
import noteDict as n

beatsPerMinute = 170
secondsPerBeat = 1/beatsPerMinute * 60

shotFired = np.array([[ n.midOctave["A"], secondsPerBeat * 0.5 ]], dtype='float')

Make a function for playing a sound file

In [ ]:
def playSong(songFile1, songfile2, buzPin1, buzPin2, ledPin1, ledPin2):

    # load in information from the song file
    song1 = np.load(songFile1).copy()
    # isolate the frequencies in the sound file for some calulations
    num_rows, numCols = song1.shape                              # get the columns and rows
    frequencyList = song1.copy()                                 # make a copy we can resize
    frequencyList = frequencyList.compress([True,False],axis=1) # resize the array so we only have the frequencies
    frequencyList = frequencyList.flatten()                     # flatten the array into 1D
    frequencyList = frequencyList[frequencyList!=0]             # remove all 0s, which would always be the minimum
    # find the minimum frequency
    minFreq = np.min(frequencyList)
    # find the maximum frequency
    maxFreq = np.max(frequencyList)

    # to get a noticible difference between different levels of dimming the LED, use the max and min frequencies to find a slope
    # value ( max change in duty cycle / max change in frequencies ), this lets us plug in the current frequency and find the adaquate
    # duty cycle value
    slope1 = 255 / (maxFreq-minFreq)
    minDuty1 = slope1 * minFreq
    
    # load in information from the second song file
    song2 = np.load(songFile2).copy()
    # isolate the frequencies in the sound file for some calulations
    num_rows, numCols = song2.shape                              # get the columns and rows
    frequencyList = song2.copy()                                 # make a copy we can resize
    frequencyList = frequencyList.compress([True,False],axis=1) # resize the array so we only have the frequencies
    frequencyList = frequencyList.flatten()                     # flatten the array into 1D
    frequencyList = frequencyList[frequencyList!=0]             # remove all 0s, which would always be the minimum
    # find the minimum frequency
    minFreq = np.min(frequencyList)
    # find the maximum frequency
    maxFreq = np.max(frequencyList)

    # to get a noticible difference between different levels of dimming the LED, use the max and min frequencies to find a slope
    # value ( max change in duty cycle / max change in frequencies ), this lets us plug in the current frequency and find the adaquate
    # duty cycle value
    slope2 = 255 / (maxFreq-minFreq)
    minDuty2 = slope2 * minFreq

    # initialize the led to be off with a frequency of 100 so it doesn't blink when it turns on
    pi.set_PWM_frequency(ledPin1,100)
    pi.set_PWM_dutycycle(ledPin1,0)
    pi.set_PWM_frequency(ledPin2,100)
    pi.set_PWM_dutycycle(ledPin2,0)

    #loop through every note in the songs
    song1Length = song1.size()
    song1NoteStart = 0
    song1Index = 0
    
    song2Length = song2.size()
    song2NoteStart = 0
    song2Index = 0
    while True:
        if song1Index >= song1Length and song2Index >= song2Length:
            break
        
        # check the first song
        # get how long the current note has been playing
        tdiff1 = time.time() - song1NoteStart
        # if the current note has finished: start the next note if there is one
        if tdiff1 - song1[song1Index][1] > 0 and song1Index < song1Length:
            pi.hardware_PWM(buzPin1, int(song1[song1Index][0]), int(0.25e6))  # set the buzzer PWM
            if song1[song1Index][0] == 0:
                pi.set_PWM_dutycycle(ledPin,0)                             # if it's a rest, don't use the slope equation otherwise it will create an error
            else:                                               
                pi.set_PWM_dutycycle(ledPin,slope1*song1[song1Index][0] - minDuty)    # set the LED duty cycle based off the slope and frequency
            # reset the start time of the current note
            song1NoteStart = time.time()
            song1Index+=1
        
        # check the second song
        # get how long the current note has been playing
        tdiff2 = time.time() - song1NoteStart
        # if the current note has finished: start the next note if there is one
        if tdiff2 - song2[song2Index][1] > 0 and song2Index < song2Length:
            pi.hardware_PWM(buzPin2, int(song2[song2Index][0]), int(0.25e6))  # set the buzzer PWM
            if song2[song2Index][0] == 0:
                pi.set_PWM_dutycycle(ledPin2,0)                             # if it's a rest, don't use the slope equation otherwise it will create an error
            else:                                               
                pi.set_PWM_dutycycle(ledPin2,slope2*song2[song2Index][0] - minDuty)    # set the LED duty cycle based off the slope and frequency
            # reset the start time of the current note
            song2NoteStart = time.time()
            song2Index+=1
    
    pi.hardware_PWM(buzPin1, 0, 0)
    pi.hardware_PWM(buzPin2, 0, 0)
    pi.set_PWM_dutycycle(ledPin1,0)
    pi.set_PWM_dutycycle(ledPin2,0)

Do any GPIO setup we'll need

In [ ]:
GPIO.setmode(GPIO.BCM)

buzPin = 18
buttonPin = 17
ledList = [19, 20, 21, 22, 23, 24]

GPIO.setup(buzPin, GPIO.OUT)
GPIO.setup(buttonPin, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
for led in ledList:
    GPIO.setup(led, GPIO.OUT)

Establish the pigpio port

In [ ]:
pi = pigpio.pi(port = 8887)

When we start things up, play the Iron Man Theme

In [ ]:
playSong('Can_You_Dig_It.npy', buzPin, ledList)

Need to be waiting for a button press. Once the button is pressed, fire a shot.
The numpy file will contain the information for the sound of the shot, and the leds will shine in accordance to the frequencies being played

In [ ]:
try:
    while(True):
        if GPIO.input(buttonPin) == 1:
            playSong('fireShot.npy', buzPin, ledList)
                
except(KeyboardInterrupt, SystemExit):
    print("Interupt!")

Stop the pigpio port and GPIO

In [ ]:
pi.stop()
GPIO.cleanup()